目标检测之Faster RCNN
===
Faster RCNN在Fast RCNN的基础上，将proposal的选取工作也放到了网络中，这样通过一个CNN网络，就可以同时生成Proposal(RPN层)，而且也可以同时进行分类和回归的训练
![image](Images/03/02/01_04_001.png)

## 4.1.总体架构
![image](Images/03/02/01_04_002.png)

## 4.2.核心网络架构
![image](Images/03/02/01_04_003.png)

## 4.3.一些解释
### 4.3.1.关于Conv Feature Map
对于获取Conv Feature Map的CNN网络，我们一般使用ZF5层网络或VGG16的13层网络这两种形式；如果是ZF5层网络，它的输出是$N\*N*256$；如果是VGG16的话它的输出就是$N*N*512$

### 4.3.2.关于gt_boxes
gt_boxes就是Group Truth Boxes，它是训练集中的true proposals的信息，一般放在annoations下面

## 4.3.3.关于整体流程
整个Faster RCNN相当于是RPN网络+Fast RCNN的集合，首先它需要训练RPN网络，然后根据训练好的RPN网络，得出proposals，然后与原始图片一同传入RoI Pooling层得到最终统一的rois，然后传入全连接层进行训练。

## 4.3.4.关于Anchor boxes
1. 我们可以看到过了CNN之后，会输出一个N\*N\*256的feature map，相当于这是一张N\*N的图，且有256通道。然后经过一个3*3卷积和两个1\*1的卷积之后开始分叉，一个输出N\*N\*18，一个输出N\*N\*36。这是因为我们在每个点上增加了9个anchor box。(这么做的原因是因为一个格子里可能包含多余一个物品)。然后我们将这些anchor box映射到原图中可以得到每个anchor在原图中的位置(然后和group truth boxes做loss就能进行位置精修了)。对于18，是2\*9得来的，因为每一个anchor都包含两个信息foreground和background，36就是4\*9得来的，每个anchor有4个坐标
2. 并不是所有生成的anchor box都需要用来做训练，那样就太多了。我们给出了两个条件来标示正标签
> - 与GT包围盒最高IoU重叠的anchor(也许IoU $\leq$ 0.7)
> - 与任意GT包围盒d额IoU $\geq$ 0.7的anchor
3. 根据条件可以知道，一个GT包围盒有可能对应多个anchor box的，那么negative labels就是与所有GT包围盒$IoU \leq 0.3$的anchor，对于既不是正标签又不是负标签的anchor box那么就直接舍弃。起初RPN提取的anchor的偏差肯定很大，所以需要backward的迭代优化

## 4.4.ROI Pooling层
ROIs Pooling顾名思义，是Pooling层的一种，而且是针对RoIs的Pooling，它的特点是输入特征图尺寸不固定，但是输出特征图尺寸固定。ROI是Region of Interest的简写，指的是在“特征图上的框”；
- 在Fast RCNN中，ROI是指Selective Search完成后得到的“候选框”在特征图上的映射；
- 在Faster RCNN中，候选框是经过RPN产生的，然后再把各个“候选框”映射到特征图上，得到ROIs。
https://github.com/SirLPS/roi_pooling
https://zhuanlan.zhihu.com/p/32404424